# Monte Carlo Simulations to Sample the Canonical Distribution

**Authors:** Dou Du, Taylor James Baird and Giovanni Pizzi

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/statistical-mechanics/monte_carlo_parabolic.ipynb

We demonstrate the Monte Carlo simulations sampling the canoical distribution in this notebook.

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Goals
* Understand how a Monte Carlo simulation samples the canonical distribution (with simple examples).
* Appreciate the effect of temperature on the simulations.
* Familiarize oneself with the role of numerical parameters (number of steps, step size).
* Gain insight into the difficulties encountered when considering low temperatures, high barriers, and achieving ergodicity.

## **Background theory** 

[More on the background theory.](./theory/theory_monte_carlo_parabolic.ipynb)

## Tasks and exercises

1. Find the approximate <code>(x, y)</code> coordinates of the minima of the potential both from the potential energy surface plot and from the peaks of the probability distribution (after running the simulation with the default parameters).
   Do this for both potentials (single and double well). Note that when you hover on the potential energy surface plot, you can also see some black isosurface lines, that can help you find where the minimum is.<br>

    <details>
    <summary style="color: red">Solution</summary>

    There is only one global minimum at <code>(x=0, y=0)</code> in the single parabolic well case.<br>

    In the double well case, two global minima are located at around <code>(x=0, y=3.13)</code> and <code>(x=0, y=-3.13)</code>.<br>
    
    In the probability distribution plot, if you run with the default variables, you should see clear peaks around the minima.<br><br>
    
    </details>

1. Select to the single-well case and use the slider to modify the temperature. What do you expect the converged average total energy to be? Is this the case?

    <details>
    <summary style="color: red">Solution</summary>
    For a single quantum well with two coordinates (x and y), we have two degrees of freedom. Therefore, by equipartition, we expect the average energy to be $2\cdot \frac {k_B T}{2}=k_B T$ (i.e., in the reduced units used here with $k_B=1$, the same numerical value as the temperature).
    As you change the temperature (try e.g. 0.1, 1 and 5), you should see that the average energy tends to reach the same value as the temperature.
    Note: in order to keep the equilibration period very short, it might be convenient to set the starting point to <code>(0, 0)</code>. Otherwise, note you can zoom in on the plot to better inspect the values.<br>
    
    You will not necessarily converge to the exact value: try to run the simulation more than once, or try to increase the simulation steps or tune the other numerical parameters (we will do this together in the tasks below).<br><br>
    
    </details>

1. Select to the single well case. Set the starting position to, e.g., (10, 10) to be out of the minimum, and the temperature to 1. Change the max move size (for example, try values around 0.1, 1 and 5). Inspect the convergence of the total energy to the converged result, both in terms of how long the equilibration takes, and the statistical error.

    <details>
    <summary style="color: red">Solution</summary>
    You should notice that, with a smaller max move size, the simulation takes a longer time to reach convergence. You can for example check how many steps it takes to get within, say, 5% of the converged value (you can zoom in, and note that the y axis is automatically rescaled).
    
    On the other hand, while a bigger move size makes the simulation equilibrate quickly, the magnitude of the fluctuations remain larger. In addition, the acceptance rate drops significantly: most moves change the system coordinates by a large amount, increasing significantly the energy of the system, and only a few of these are statistically accepted. The acceptance rate is a typical parameter that one should monitor and tune in a simulation; if it is too large it indicates that we are accepting almost all moves (and probably then the move is small and we are far from the minimum); if it is too small, we are probably moving too much at every step; as a result the simulation becomes much more expensive (it has to perform many calculations and reject most of them) and the average quantities are affected by larger errors.<br><br>
    
    </details>

1. Let us now investigate the double-well potential, with low temperature and small max move size. You can set the starting position to, e.g., (10, 10), the temperature to 0.5, and the max move size to 0.1. Run the simulation a few times. How many minima does the simulation typically samples? What happens if you increase the max move size? What happens for higher temperatures?

    <details>
    <summary style="color: red">Solution</summary>
    When the temperature is small, the probability to jump between the two wells across the barrier is very low, and might never happen in the simulation. In this case the simulation is not ergodic, because the distribution obtained weighting the phase-space points (that would occupy identically the two wells) is different from the time average over the finite simulation (that typically sees only one well).
    
    By starting at <code>(10, 10)</code> and using a small step, we typically end up almost always in the closest of the two wells. If we start at the top of the double well (starting coordinates <code>(0, 0)</code>), with a small temperature (1 or smaller) and small max move size, each simulation will randomly fall in one of the two wells and be trapped there.
    
    If you increase the move size, the fluctuations become larger and one gets a better representation of the canonical distribution (e.g. for a max move size of 10) at the expense of larger fluctuations in the averaged quantities). In addition, most moves are rejected because the majority of them bring the system to a much higher energy.
    
    As the temperature increases, the thermal fluctuations increase and there is a higher probability that they allow us to visit both wells, if a long-enough simulation is considered.<br><br>
    
    </details>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Interactive visualization
(be patient, it might take a few seconds to load)

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
import math
from plotly.subplots import make_subplots
from ipywidgets import FloatSlider, IntSlider, FloatLogSlider, Button
from ipywidgets import Output, HBox, VBox, IntProgress, Dropdown, Layout, Label

In [ ]:
#all the widget components

description_width = '120px'

temp_slider = FloatSlider(min=0.1, max=5, value=1., continuous_update=False, description="Temperature", style={'description_width': description_width})
move_size_slider = FloatLogSlider(min=-1, max=1, value=1., continuous_update=False, description="Max move size", style={'description_width': description_width})
num_iterations_slider = IntSlider(min=100000, max=1000000, value=100000, continuous_update=False, description="Num iterations", style={'description_width': description_width})
run_button = Button(description="Run Monte-Carlo")
run_button.style.button_color = 'green'
traces_button = Button(description="Show Traces")
x_slider = FloatSlider(value=4.0, min=-10.0, max=10.0, continuous_update=False, description="Initial pos. x", style={'description_width': description_width})
y_slider = FloatSlider(value=4.0, min=-10.0, max=10.0, continuous_update=False, description="Initial pos. y", style={'description_width': description_width})
potential_dropdown = Dropdown(options=[('single parabolic well',1), ('double well',2)],
                              value = 1, description='Potential:', layout=Layout(width='250px'))
label1 = Label(value='(only shows 300 uniformly sampled steps in the whole simulation)')

In [ ]:
A1 = 1. # potential curvature first axis
A2 = 1.e-1 # potential curvature second axis

# Starting point of our MC, for the two coordinates
starting_x1 = 9.0
starting_x2 = 9.0

def get_energy(x, y):
    """Compute the potential energy according to the dropbox."""
    if potential_dropdown.value == 1:
        return 0.5 * x**2 + 0.5 * 4.0 * y**2
    elif potential_dropdown.value == 2:
        return 0.5 * A1 * x**2 + 0.5 * A2 * (y**4 - 20 * y**2)

def get_property(x1, x2):
    # I can do something else, I decide to sample the energy of one of the two coordinates
    #return 0.5 * A2 * (x2**4 - 4 * x2**2)
    
    # I just return the second coordinate, to see its average
    return x2

In [ ]:
x, y = np.mgrid[-10:10:100j, -10:10:100j]
z = get_energy(x, y)

# figure one to show the potential energy surface and current position
fig1 = go.FigureWidget(data=[go.Surface(z=z, x=x, y=y, opacity=0.5),
                             go.Scatter3d(x=[starting_x1], 
                                          y=[starting_x2], 
                                          z=[get_energy(starting_x1, starting_x2)], 
                                          mode='markers+text',
                                          text=['Current position'],
                                          marker=dict(size=8, color='red'))])

def on_potential_change(c):
    """Callback function for the change of the dropdown."""
    fig1.data[0].z = get_energy(x, y)
    fig1.data[1].x = [x_slider.value]
    fig1.data[1].y = [y_slider.value]
    fig1.data[1].z = [get_energy(x_slider.value, y_slider.value)]
    
potential_dropdown.observe(on_potential_change)

In [ ]:
# figure 2 is the histogram of probabilities for the (x,y) coordinates, displayed as a heatmap.
fig2 = go.FigureWidget()

fig2.add_trace(go.Scatter(
    x=[starting_x1],
    y=[starting_x2],
    mode='lines+markers',
    visible = False,
    showlegend=False,
    marker=dict(
        symbol='circle',
        opacity=0.7,
        color='white',
        size=5,
        line=dict(width=1),
    ),
))

fig2.add_trace(go.Histogram2d(
    x=[],
    y=[],
    histnorm='probability',
    autobinx = False,
    xbins=dict(start=-10, end=10, size=0.1),
    autobiny = False,
    ybins=dict(start=-10, end=10, size=0.1),
    colorscale=[[0, 'rgb(12,51,131)'], [0.25, 'rgb(10,136,186)'], [0.6, 'rgb(242,211,56)'], 
                [0.75, 'rgb(242,143,56)'], [1, 'rgb(217,30,30)']]
))

fig1.update_layout(title='Potential energy surface', autosize=False,
                  width=420, height=420,
                  margin=dict(l=10, r=10, b=30, t=30))

fig2.update_layout(title='Probability histogram',
    xaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
    yaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
    xaxis_title="x",
    yaxis_title="y",
    autosize=False,
    height=420,
    width=430,
    margin=dict(l=30, r=30, b=60, t=60),
    hovermode='closest',

)

fig3 = go.FigureWidget()

fig3.add_trace(go.Scatter(x=[], y=[], mode='lines',
    name='Total energy',
    line=dict(color='red', width=2),
    connectgaps=True,
))

fig3.update_layout(
    height = 200,
    width = 530,
    xaxis_title = 'Monte-Carlo move',
    yaxis_title = 'Total energy',
    margin=dict(l=30, r=30, b=10, t=10),
)

display(HBox([fig1, fig2]))

In [ ]:
def run_mc(move_size, num_iterations, temp):
    """
    :param move_size: max move size for the two axes x1 and x2.
        In principle, the move size could/should be different for x1 and x2, here I choose the same for simplicity!
        
    :param num_iterations: total number of Monte-Carlo iterations
    """
    x1 = starting_x1
    x2 = starting_x2
    energy = get_energy(x1, x2)

    all_x1 = [x1]
    all_x2 = [x2]
    all_properties = [get_property(x1, x2)]
    all_energies = [get_energy(x1, x2)]
    count_accepted = 0
    count_refused = 0

    for iter_cnt in range(num_iterations):
        # shift by a random value between -move_size and + move_size
        new_x1 = x1 + ((random.random() - 0.5) * 2) * move_size
        new_x2 = x2 + ((random.random() - 0.5) * 2) * move_size
        new_energy = get_energy(new_x1, new_x2)

        if new_energy < energy:
            accepted = True
        else:
            # boltzmann_k = 1 in these units
            probability = math.exp(-(new_energy - energy)/temp)
            # random.random() is a random number between 0 and 1
            # also probability is between 0 and 1
            # so if I accept only if random.random() is < probability,
            # I'm accepting the move with 'probability' probability
            accepted = random.random() < probability

        if accepted:
            x1 = new_x1
            x2 = new_x2
            energy = new_energy
            count_accepted += 1
        else:
            count_refused += 1
        all_properties.append(get_property(x1, x2))
        all_energies.append(energy)
        all_x1.append(x1)
        all_x2.append(x2)
        
    average_integrated_energy = np.cumsum(all_energies) / (1 + np.arange(len(all_energies)))
    average_property = np.cumsum(all_properties) / (1 + np.arange(len(all_properties)))

    return count_accepted, count_refused, average_integrated_energy, average_property, all_x1, all_x2

In [ ]:
controls = VBox([x_slider, y_slider, potential_dropdown, temp_slider, move_size_slider, num_iterations_slider])

display(HBox([controls, fig3]))
display(run_button, HBox([traces_button, label1]))

In [ ]:
def change_init_position(c):
    """Callback function for the change of the x, y sliders."""
    global starting_x1, starting_x2
    starting_x1 = x_slider.value
    starting_x2 = y_slider.value
    fig1.data[1].x = [x_slider.value]
    fig1.data[1].y = [y_slider.value]
    fig1.data[1].z = [get_energy(x_slider.value, y_slider.value)]
    
x_slider.observe(change_init_position, names='value')
y_slider.observe(change_init_position, names='value')

In [ ]:
def interactive_plot(button):
    """Callback function for the run button."""
    global all_x1, all_x2
    temp = temp_slider.value
    move_size=move_size_slider.value
    num_iterations=num_iterations_slider.value
    
    run_button.disabled = True
    run_button.style.button_color = 'red'
    run_button.description = "Running..."
    traces_button.disabled = True
    try:
        count_accepted, count_refused, average_integrated_energy, average_property, all_x1, all_x2 = run_mc(
            move_size=move_size, num_iterations=num_iterations, temp=temp)
        
        fig1.data[1].x = [all_x1[-1]]
        fig1.data[1].y = [all_x2[-1]]
        fig1.data[1].z = [get_energy(all_x1[-1], all_x2[-1])]
    
        snum = int(len(all_x1)/300)
        fig2.data[0].x = all_x1[::snum]
        fig2.data[0].y = all_x2[::snum]
        
        fig2.data[1].x = all_x1
        fig2.data[1].y = all_x2
        
        skip_start = 10000
        
        fig3.data[0].x = np.arange(len(average_integrated_energy) - skip_start)
        fig3.data[0].y = average_integrated_energy[skip_start:]
    finally:
        run_button.disabled = False
        traces_button.disabled = False
        run_button.style.button_color = 'green'
        run_button.description="Run Monte-Carlo"
        
run_button.on_click(interactive_plot)

In [ ]:
def toggle_show_traces(button):
    """Callback function for the show traces button."""
    fig2.data[0].visible ^= True
    if fig2.data[0].visible:
        traces_button.description = "Hide traces"
    else:
        traces_button.description = "Show traces"
    
traces_button.on_click(toggle_show_traces)


def show_current_point(trace, points, selector):
    for i in points.point_inds:
        fig1.data[1].x = [all_x1[i]]
        fig1.data[1].y = [all_x2[i]]
        fig1.data[1].z = [get_energy(all_x1[i], all_x2[i])]
        

        fig3.data[0].on_click(show_current_point)

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Legend
(How to use the interactive visualization)

### Potential energy surface
The left panel shows the potential energy surface (PES).

You can rotate the surface by clicking and dragging.
The `z` value indicates the value of the potential energy.

When the mouse is on the potential energy surface, you will also see a black isoline, which you can use to find the global minimum.

### Controls
All the control widgets are on the bottom left.

You can choose the initial `x` and `y` coordinates of the simulation.
You can choose between two simple potentials: a single parabolic well, and a double well.
The second can help understanding how the algorithm can overcome energy barriers.

You can also tune a number of system and numerical parameters: the simulation temperature,
the maximum move size, and the number of the simulation steps.

### Running the simulation and resulting plots
You can start the simulation by clicking the `Run Monte Carlo` button.
This might take a few seconds before it completes. All plots will then be updated.

The histogram of the canonical distribution as sampled by the simulation will be shown as a heatmap on the top right.
By clicking the `Show traces` button, it will show 300 (evenly distributed) points on the heatmap.

The total energy as a function of the step number is shown on the bottom right panel.
10000 steps from the beginning of the simulation are skipped for the total energy figure as they are considered as an initial thermalization; note, however, that if you start in a point with very low probability (i.e., far away from the minimum), the simulation might require many more steps to thermalize.